# Retail Strategy and Analytics

---

# Data Preparation

## Load required libraries

Note that you will need to install these libraries if you have never used these
before.

In [ ]:
# Data manipulation
import numpy as np
import pandas as pd
import datetime as dt

# Data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Shows plots in jupyter notebook
%matplotlib inline

# Set plot style
sns.set(color_codes=True)

# Ignore warning
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('Matplotlib', mpl.__version__)
print('Seaborn', sns.__version__)

---
## Load data with pandas

In [ ]:
# Load transaction and customer data
transactionData = pd.read_excel('https://github.com/sabirinID/Quantium-Data-Analytics-Virtual-Internship/raw/main/Files/1-Data%20Preparation%20and%20Customer%20Analytics/QVI_transaction_data.xlsx')
customerData = pd.read_csv('https://github.com/sabirinID/Quantium-Data-Analytics-Virtual-Internship/raw/main/Files/1-Data%20Preparation%20and%20Customer%20Analytics/QVI_purchase_behaviour.csv')

---
## Data description

### Transaction data

| Feature Name | Description |
|--------------|-------------|
| DATE | Date of the transaction |
| STORE_NBR | Identification number of the store where the transaction took place |
| LYLTY_CARD_NBR | Loyalty card number of the customer |
| TXN_id | Identification number of the transaction |
| PROD_NBR | Identification number of the product |
| PROD_NAME | Name of the product |
| PROD_QTY | Quantity of the product purchased in a single transaction |
| TOT_SALES | Total sales from the transaction |

### Customer data

| Feature Name | Description |
|--------------|-------------|
| LYLTY_CARD_NBR | Loyalty card number of the customer |
| LIFESTAGE | Life stage of the customer |
| PREMIUM_CUSTOMER | Indicates whether the customer is a premium customer or not |

---
## Exploratory data analysis

The first step in any analysis is to first understand the data. Let's take a look
at each of the datasets provided.

### Examining transaction data

In [ ]:
transactionData.shape

In [ ]:
transactionData.columns

In [ ]:
# # Rename columns
# transactionData.rename(columns={
#     'DATE'          : 'transaction_date',
#     'STORE_NBR'     : 'store_id',
#     'LYLTY_CARD_NBR': 'loyalty_card_id',
#     'TXN_ID'        : 'transaction_id',
#     'PROD_NBR'      : 'product_id',
#     'PROD_NAME'     : 'product_name',
#     'PROD_QTY'      : 'product_quantity',
#     'TOT_SALES'     : 'total_sales'
#     }, inplace=True)

# Look at the first 10 rows
transactionData.head(10)

- Let's check if columns we would expect to be numeric are in numeric form and date
columns are in date format.

#### Examine transaction data

In [ ]:
transactionData.info()

- We can see that the `DATE` column is in an integer format. Let's change this to a
date format.
- We should check that we are looking at the right products by examining `PROD_NAME`.

#### Convert `DATE` column to a date format

- A quick search online tells us that CSV and Excel integer dates begin on 30
Dec 1899.

In [ ]:
# Convert 'DATE' column to datetime format
transactionData['DATE'] = pd.to_datetime(transactionData['DATE'], origin='1899-12-30', unit='D')

transactionData['DATE'].describe()

#### Examine `PROD_NAME`

- Generate a summary of the `PROD_NAME` column.

In [ ]:
transactionData['PROD_NAME'].describe()

In [ ]:
transactionData['PROD_NAME'].nunique()

In [ ]:
transactionData['PROD_NAME'].value_counts(normalize=True).round(5)

- Looks like we are definitely looking at potato chips but how can we check that
these are all chips? We can do some basic text analysis by summarising the
individual words in the product name, `PROD_NAME`.
- Examine the words in `PROD_NAME` to see if there are any incorrect entries such as products that are not chips

In [ ]:
# Extract product words
productWords = transactionData['PROD_NAME'].str.split().apply(pd.Series, 1).stack()
productWords.index = productWords.index.droplevel(-1)
productWords.name = 'words'

- As we are only interested in words that will tell us if the product is chips or
not, let's remove all words with digits and special characters such as '&' from our
set of product words.

In [ ]:
# Remove digits
productWords = productWords.str.replace(r'\d+', '', regex=True)

# Remove special characters
productWords = productWords.str.replace(r'[^\w\s]', '', regex=True)

# Remove empty spaces
productWords = productWords[productWords.str.replace(' ', '').str.isalpha()]

- Let's look at the most common words by counting the number of times a word
appears and sorting them by this frequency in order of highest to lowest frequency.

In [ ]:
# Count the frequency of each word
wordCounts = productWords.value_counts().reset_index()
wordCounts.columns = ['words', 'frequency']

# Sort the distinct words by frequency of occurrence
wordCounts = wordCounts.sort_values(by='frequency', ascending=False)

In [ ]:
# Remove all words that are alphabets, conjunctions, and prepositions
wordCounts = wordCounts[~wordCounts['words'].isin(['g', 'G', 'And', 'Of'])]

In [ ]:
wordCounts.head(13)

- There are `Salsa` products in the dataset but we are only interested in the chips
category, so let's remove these.

#### Remove `Salsa` products

In [ ]:
# Remove salsa products
transactionData['Salsa'] = transactionData['PROD_NAME'].str.contains('Salsa', case=False)
transactionData = transactionData[~transactionData['Salsa']]
transactionData.drop('Salsa', axis=1, inplace=True)

- Next, we can use `describe()` to check summary statistics such as mean, min and max
values for each feature to see if there are any obvious outliers in the data and if
there are any nulls in any of the columns (`NA's : number of nulls` will appear in
the output if there are any nulls).

#### Summarise the data to check for nulls and possible outliers

In [ ]:
transactionData.describe(include='all')

- There are no nulls in the columns but product quantity, `PROD_QTY`, appears to have an outlier which we should investigate further. Let's investigate further the case where 200
packets of chips are bought in one transaction.

#### Filter the dataset to find the outlier

In [ ]:
transactionData[transactionData['PROD_QTY'] == 200]

- There are two transactions where 200 packets of chips are bought in one transaction
and both of these transactions were by the same customer.
- Let's see if the customer has had other transactions.

In [ ]:
transactionData[transactionData['LYLTY_CARD_NBR'] == 226000]

- It looks like this customer has only had the two transactions over the year and is
not an ordinary retail customer. The customer might be buying chips for commercial
purposes instead. We'll remove this loyalty card number from further analysis.

#### Filter out the customer based on the loyalty card number

In [ ]:
# Get the indices of the rows where 'LYLTY_CARD_NBR' is 226000
indices = transactionData[transactionData['LYLTY_CARD_NBR'] == 226000].index

# Drop the rows
transactionData.drop(indices, inplace=True)

#### Re-examine transaction data

In [ ]:
transactionData['PROD_QTY'].describe()

- That's better. Now, let's look at the number of transaction lines over time to see
if there are any obvious data issues such as missing data.

#### Count the number of transactions by `DATE`

In [ ]:
transactionData.groupby('DATE')['TXN_ID'].count()

#### Create a summary of transaction count by `DATE`.

In [ ]:
transactionData.groupby('DATE')['TXN_ID'].count().describe()

- There's only 364 rows, meaning only 364 dates which indicates a missing date. Let's
create a sequence of dates from 1 Jul 2018 to 30 Jun 2019 and use this to create a
chart of number of transactions over time to find the missing date.

#### Create a sequence of dates and join this the count of transactions by `DATE`

In [ ]:
# Create a date range from 1 July 2018 to 30 June 2019
date_range = pd.date_range(start='2018-07-01', end='2019-06-30')

# Convert the date range to a DataFrame
date_df = pd.DataFrame(date_range, columns=['DATE'])

# Set the 'DATE' column as the index
date_df.set_index('DATE', inplace=True)

# Join the 'transactionData' DataFrame with the 'date_df' DataFrame
# using a left join. This will fill in missing days with NaN values.
transaction_date = transactionData.groupby('DATE').size()
transaction_date.name = 'TXN_SIZE'
joined_df = date_df.join(transaction_date, how='left')

# Reset the index
joined_df.reset_index(inplace=True)

In [ ]:
joined_df.head(3)

In [ ]:
joined_df.tail(2)

In [ ]:
joined_df[joined_df['TXN_SIZE'].isna() == True]

- There were no transactions on 25 Dec 2018.
- This could be due to a variety of reasons, such as holidays, weekends, or the absence of any business activity on that specific day. It's also possible that there were no data entries for that day, which could be due to data collection, data entry, or data cleaning issues.

In [ ]:
# Fill in the missing values with 0
joined_df.fillna(0, inplace=True)

#### Plot transactions over time

In [ ]:
from matplotlib.dates import DateFormatter

txn_by_day = joined_df.copy()

# Create a figure
fig, ax = plt.subplots(figsize=(16, 9))
ax.plot(txn_by_day['DATE'], txn_by_day['TXN_SIZE'])
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))

plt.title('Transactions over time', fontsize=20)
plt.xlabel('\nDay', fontsize=18)
plt.ylabel('Number of transactions\n', fontsize=18)
plt.xticks(fontsize=16, rotation=90)
plt.yticks(fontsize=16)
plt.ylim(0, 875)

plt.show()

- We can see that there is an increase in purchases in December and a break in late
December. Let's zoom in on this.

#### Filter to December and look at individual days

In [ ]:
# Filter data to December
dec_data = txn_by_day.loc[txn_by_day['DATE'].dt.month == 12] # December

# Create a new figure
fig, ax = plt.subplots(figsize=(16, 9))
ax.plot(dec_data['DATE'], dec_data['TXN_SIZE'])
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m-%d'))      # Show individual days
plt.title('Transactions in December', fontsize=20)
plt.xlabel('\nDay', fontsize=18)
plt.ylabel('Number of transactions\n', fontsize=18)
plt.xticks(fontsize=16, rotation=90)
plt.yticks(fontsize=16)
plt.ylim(0, 875)

plt.show()

- We can see that the **increase** in sales occurs in the lead-up to Christmas and that
there are zero sales on Christmas day itself. This is due to shops being closed on
Christmas day.
- Now that we are satisfied that the data no longer has outliers, we can move on to
creating other features such as brand of chips or pack size from `PROD_NAME`. We will
start with pack size.

#### Pack size

- We can work this out by taking the digits that are in `PROD_NAME`.

In [ ]:
# Extract pack size
transactionData['PACK_SIZE'] = transactionData['PROD_NAME'].str.extract('(\d+)').astype(int)

- Let's check if the pack sizes look sensible.

In [ ]:
transactionData['PACK_SIZE'].sort_values(ascending=False).unique()

- The largest size is 380g and the smallest size is 70g - seems sensible!
- Let's plot a histogram of `PACK_SIZE` since we know that it is a categorical
variable and not a continuous variable even though it is numeric.

#### Plot a histogram showing the number of transactions by `PACK_SIZE`

In [ ]:
# Create a histogram of 'PACK_SIZE'
plt.figure(figsize=(16, 9))
n, bins, patches = plt.hist(transactionData['PACK_SIZE'], bins=10, color='gray')

# Find the bin with the highest frequency
max_bin = np.argmax(n)

# Change the color of the highest bin to orange
patches[max_bin].set_facecolor('tab:orange')

# Add annotations for each bin
for i in range(len(n)):
    plt.annotate(round(n[i]), (bins[i], n[i]), textcoords='offset points',
                 xytext=(40, 10), ha='center', va='center')

plt.title('Transactions by pack size', fontsize=20)
plt.xlabel('\nPack Size (gram)', fontsize=18)
plt.ylabel('Number of transactions\n', fontsize=18)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.ylim(0, 115000)

plt.show()

- Pack sizes created look reasonable.
- Now to create brands, we can use the first word in `PROD_NAME` to work out the brand
name.

#### Brands

- Create a column which contains the brand of the product, by
extracting it from the product name, `PROD_NAME`.

In [ ]:
# Extract brand
transactionData['BRAND'] = transactionData['PROD_NAME'].str.extract('([A-Z][^A-Z]*)')

# Convert to integer only if the string is numeric
transactionData['BRAND'] = transactionData['BRAND'].apply(lambda x: int(x) if x.isnumeric() else x)

- Check the results look reasonable.

In [ ]:
transactionData['BRAND'].sort_values().unique()

- Some of the brand names look like they are of the same brands - such as Red and
RRD, which are both Red Rock Deli chips. Let's combine these together.

#### Clean brand names

In [ ]:
# Rename brand
transactionData['BRAND'].replace('R', 'Red ', inplace=True)

#### Check again



In [ ]:
transactionData['BRAND'].sort_values().unique()

### Examining customer data

- Now that we are happy with the transaction dataset, let's have a look at the
customer dataset.
- Do some basic summaries of the dataset, including distributions of
any key columns.

In [ ]:
customerData.shape

In [ ]:
customerData.columns

In [ ]:
# # Rename columns
# customerData.rename(columns={
#     'LYLTY_CARD_NBR'  : 'loyalty_card_id',
#     'LIFESTAGE'       : 'customer_life_stage',
#     'PREMIUM_CUSTOMER': 'premium_customer'
#     }, inplace=True)

# Look at the first 10 rows
customerData.head(10)

#### Examine customer data

In [ ]:
customerData.info()

#### Merge transaction data to customer data

In [ ]:
# Merge transactionData and customer_data
data = pd.merge(transactionData, customerData, on='LYLTY_CARD_NBR', how='left')
data.head(2)

- As the number of rows in `data` is the same as that of `transactionData`, we can be
sure that no duplicates were created. This is because we created `data` by setting
`how='left'` (in other words, a left join) which means take all the rows in
`transactionData` and find rows with matching values in shared columns and then
joining the details in these rows to the first mentioned table.
- Let's also check if some customers were not matched on by checking for nulls.

#### Check for missing customer details

- See if any transactions did not have a matched customer.

- Great, there are no nulls! So all our customers in the transaction data has been accounted for in the customer dataset.

#### Export the dataset

In [ ]:
# Save to CSV
data.to_csv('QVI_data.csv', index=False)

- Data exploration is now complete!

---

## Data analysis on customer segments

Now that the data is ready for analysis, we can define some metrics of interest to the client:
- Who spends the most on chips (total sales), describing customers by lifestage and
how premium their general purchasing behaviour is
- How many customers are in each segment
- How many chips are bought per customer by segment
- What's the average chip price by customer segment

We could also ask our data team for more information. Examples are:
- The customer's total spend over the period and total spend for each transaction to understand what proportion of their grocery spend is on chips
- Proportion of customers in each customer segment overall to compare against the mix of customers who purchase chips

Let's start with calculating total sales by `LIFESTAGE` and `PREMIUM_CUSTOMER` and plotting the split by these segments to describe which customer segment contribute most to chip sales.

#### Total sales by `LIFESTAGE` and `PREMIUM_CUSTOMER`

- Calculate the summary of sales by those dimensions and create a
plot.

- Sales are coming mainly from Budget - older families, Mainstream - young singles/couples, and Mainstream - retirees.
- Let's see if the higher sales are due to there being more customers who buy chips.

#### Number of customers by `LIFESTAGE` and `PREMIUM_CUSTOMER`

- Calculate the summary of number of customers by those dimensions and create a plot.

- There are more Mainstream - young singles/couples and Mainstream - retirees who buy chips. This contributes to there being more sales to these customer segments but this is not a major driver for the Budget - Older families segment.
- Higher sales may also be driven by more units of chips being bought per customer. Let's have a look at this next.

#### Average number of units per customer by `LIFESTAGE` and `PREMIUM_CUSTOMER`

- Calculate and plot the average number of units per customer by those two dimensions.

- Older families and young families in general buy more chips per customer.
- Let's also investigate the average price per unit chips bought for each customer segment as this is also a driver of total sales.

#### Average price per unit by `LIFESTAGE` and `PREMIUM_CUSTOMER`

- Calculate and plot the average price per unit sold (average sale
price) by those two customer dimensions.

- Mainstream midage and young singles and couples are more willing to pay more per packet of chips compared to their budget and premium counterparts. This may be due to premium shoppers being more likely to buy healthy snacks and when they buy chips, this is mainly for entertainment purposes rather than their own consumption. This is also supported by there being fewer premium midage and young singles and couples buying chips compared to their mainstream counterparts.

- As the difference in average price per unit isn't large, we can check if this difference is statistically different.

#### Perform an independent t-test between mainstream vs premium and budget midage and young singles and couples

- Perform a t-test to see if the difference is significant.

- The t-test results in a p-value of XXXXXXX, i.e. the unit price for mainstream, young and mid-age singles and couples [ARE / ARE NOT] significantly higher than that of budget or premium, young and midage singles and couples.

## Deep dive into specific customer segments for insights

We have found quite a few interesting insights that we can dive deeper into.

We might want to target customer segments that contribute the most to sales to retain them or further increase sales. Let's look at `Mainstream` - young singles/couples. For instance, let's find out if they tend to buy a particular brand of chips.

#### Deep dive into `Mainstream`, young singles/couples

- Work out of there are brands that these two customer segments prefer more than others. We could use a technique called affinity analysis or a-priori analysis.

We can see that :

[INSIGHTS]

Let's also find out if our target segment tends to buy larger packs of chips.

#### Preferred pack size compared to the rest of the population

[INSIGHTS]